* Pecu https://github.com/pecu/PyTorch_CSX/blob/master/01_PyTorch_Basic/01_TensorDef.ipynb
* 作者: 莫烦 https://morvanzhou.github.io/tutorials/machine-learning/torch/2-02-variable/

In [38]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Tensor / numpy array
* PyTorch 的張量本質上和numpy的數組是一樣的：張量是一個ñ維數組，並且PyTorch定義了關於張量的很多操作,他們可以互相轉換。

In [22]:
list1 = [[3,2,1], [4,5,6]]

# list > numpy:
arr1 = np.array(list1)
print(arr1)


# list > pytorch-tensor
tr1 = torch.FloatTensor(list1)
print(tr1)

[[3 2 1]
 [4 5 6]]

 3  2  1
 4  5  6
[torch.FloatTensor of size 2x3]



In [23]:
# numpy > pytorch-tensor
tr2 = torch.from_numpy(arr1) # dtype same as numpy
print(tr2)


# pytorch-tensor > numpy
arr2 = tr1.numpy()
print(arr2)


 3  2  1
 4  5  6
[torch.LongTensor of size 2x3]

[[ 3.  2.  1.]
 [ 4.  5.  6.]]


# Variable
* 在Torch中的變量就是一個存放會變化的值的地理位置。 裡面的值會不停的變化，如果用一個變量進行計算，那結果也是一個同類型的變量。

In [24]:
# 先產生數值
tensor = torch.FloatTensor([[1,2],[3,4]])

# 把值放到變數籃子裡： requires_grad表示是否參與誤差反向傳播 & 要不要計算梯度
variable = Variable(tensor, requires_grad=True)
variable

Variable containing:
 1  2
 3  4
[torch.FloatTensor of size 2x2]

* 變量計算時，它會在背後默默搭建計算圖。可以將所有的計算步驟（節點）都連接起來，最後進行誤差反向傳遞的時候，一次性將所有變量裡面的修改幅度（梯度）都計算出來。

In [27]:
v_out = torch.mean(variable * variable)   
print(v_out)

Variable containing:
 7.5000
[torch.FloatTensor of size 1]



In [28]:
v_out.backward() 
print(variable.grad)    # 初始 Variable 的梯度
# v_out = (1/4) * sum(variable * variable)  =>  v_out 對 variable 微分 ＝ 1/2 (variable) #次方下乘,次方減 1

Variable containing:
 0.5000  1.0000
 1.5000  2.0000
[torch.FloatTensor of size 2x2]



In [29]:
# 變數很多運算做不了，可以用 variable.data 取出數值（變成 Tensor）
print(variable.data)  


 1  2
 3  4
[torch.FloatTensor of size 2x2]



# Embedding

In [80]:
# 有一組字串
'I love machine learning !'

# 字典化(我們只能處理數字)
word_to_ix = {'I': 0, 'love': 1, 'machine':2, 'learning':4, '!':5}  

# 張量化
word_tensor = torch.LongTensor([word_to_ix["I"]])

# 裝進 Variable
word_var = Variable(word_tensor)

# 把這個 Variable 變成一串數字 
embeds = nn.Embedding(1, 10)
word_embed = embeds(word_var)

print(word_embed)

Variable containing:
-0.8779  0.3539 -0.1339 -0.6063  0.4897 -1.3214 -0.7120 -0.8863 -1.9487  0.6305
[torch.FloatTensor of size 1x10]



In [81]:
# 一次將一句話向量化(裝在變數裡)
word_tensor = torch.LongTensor([ word_to_ix["I"],
                                 word_to_ix["love"],
                                 word_to_ix["machine"],
                                 word_to_ix["learning"],
                                 word_to_ix["learning"]
                               ])

# 裝進 Variable
word_var = Variable(word_tensor)

# 把這個 Variable 變成一串數字 
embeds = nn.Embedding(5, 10)
word_embed = embeds(word_var)

print(word_embed)

Variable containing:
-1.2273 -2.2323  1.0110  0.6407  0.4141 -1.3433 -0.1592 -0.4565 -2.3250 -0.3184
-0.1861 -0.1434 -1.1034  1.5606 -0.3592  0.6865  0.0501  0.1793  0.0512  0.7916
 0.8520  1.1350 -0.5953 -0.0205  0.4180  0.6729 -0.6601 -0.9819 -0.0521 -0.3206
-0.1691 -0.8537  0.2830 -0.5726  0.0816 -0.5576 -0.7978 -0.4091 -1.1161 -0.9478
-0.1691 -0.8537  0.2830 -0.5726  0.0816 -0.5576 -0.7978 -0.4091 -1.1161 -0.9478
[torch.FloatTensor of size 5x10]



# NLP 名詞

* one-hot representation : <br>
最直觀，也是最常用的詞表示方法, 這方法把每個詞表示為一個很長的向量。這個向量的維度是詞表大小，其中絕大多數元素為0，只有一個維度的值為1，這個維度就代表了當前的詞。<br>
ex: 上面的 'I' =    [1, 0, 0, 0, 0]
ex: 上面的 'love' = [0, 1, 0, 0, 0]


* 統計語言模型（N-gram 為例）：<br>
one-hot representation 無法表示上下文關係，因此有一種統計的表示方法，就是從訓練資料計算 每N個字 出現下一個字的機率。<br>
ex: 上面的 'I love machine learning !' => I 後面出現 love 是 1/5..等。
可以想像，講機器學習的文章，machine 後面 出現learning 的機率會很高。


* 詞的分佈表示( word embedding 為例):<br>
透詞向量化以及神經網絡訓練後，可以對目標詞之間的上下關係進行建模，讓每個詞的向量依照loss去調整。<br>
可以解決one-hot 維度過大的缺點（幾個詞就幾維），將原來稀疏的巨大維度壓縮嵌入到一個更小維度的空間。


## 神經網絡語言模型與word2vec

* 通過神經網絡訓練語言模型可以得到詞向量，那麼，究竟有哪些類型的神經網絡語言模型呢？
1. Neural Network Language Model ，NNLM
2. Log-Bilinear Language Model， LBL
3. Recurrent Neural Network based Language Model，RNNLM
4. Collobert 和 Weston 在2008 年提出的 C&W 模型
5. Mikolov 等人提出了 CBOW（ Continuous Bagof-Words）和 Skip-gram 模型


* word2vec: <br>
word2vec 是 Google開發的開源工具，來實現CBOW 和 Skip-gram語言模型，把詞彙向量化。<br>

Word2Vec實際上是兩種不同思想實現的：CBOW（連續字袋）和Skip-gram。
CBOW的目標是根據上下文來預測當前詞語的概率，並且上下文所有的詞對當前詞出現概率的影響的權重是一樣的，因此叫連續的袋字樣模型。如在袋子中取詞，取出數量足夠的詞就可以了，至於取出的先後順序是無關緊要的。
skip-gram 剛好相反：根據當前詞語來預測上下文的概率。
這兩種方法都利用人工神經網絡作為它們的分類算法。起初每個單詞都是一個隨機N維向量。經過訓練之後，該算法利用CBOW或者Skip-gram的方法獲得了每個單詞的最優向量。

In [93]:
# Applies a linear transformation to the incoming data: y = Ax + b
m = nn.Linear(5, 6, bias=True)
input = Variable(torch.FloatTensor([1,2,3,4,5]))
output = m(input)
print(output)

Variable containing:
-2.2609
 0.0278
 1.5584
-2.4007
-1.7412
-1.0453
[torch.FloatTensor of size 6]



In [95]:
# torch.bmm(多矩陣相乘)
batch1 = torch.randn(10, 3, 4) # 10 個 3X4
batch2 = torch.randn(10, 4, 5) # 10 個 4X5
res = torch.bmm(batch1, batch2)
res.size()# 10 個 3X5

torch.Size([10, 3, 5])